In [3]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [14]:
file = pd.read_csv('Sales.csv')

In [15]:
#Create interactive dataframe pipeline
ifile = file.interactive()

In [24]:
#Create panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=2011, end=2016, step=1)
year_slider

IntSlider(end=2016, name='Year slider', start=2011, value=2011)

In [25]:
#Radio buttons for revenue measures
yaxis_revenue = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options = ['Revenue', 'Profit', 'Cost'],
    button_type='success'
)

In [39]:
Countrys = ['France', 'Germany', 'Canada', 'United Kingdom', 'United States', 'Australia']

#Create a pipeline to link the widgets to the dataframe
rev_pipeline = (
    ifile.loc[(ifile.Year <= year_slider) & (ifile.Country.isin(Countrys))]
    .groupby(['Country', 'Year'])[yaxis_revenue].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

In [31]:
rev_pipeline

In [40]:
#Creating line plot
revenue_plot = rev_pipeline.hvplot(x = 'Year', by='Country', y=yaxis_revenue, line_width=2, title='Revenue gained based on country')
revenue_plot

In [41]:
revenue_table = rev_pipeline.pipe(pn.widgets.Tabulator, pagingation='remote', page_size=10, sizing_mode='stretch_width')
revenue_table

In [51]:
#Creating a dashboad for visualization
#We will use an already available template on panel

template = pn.template.FastListTemplate(
    title='Bike Sales Dashboad',
    sidebar=[pn.pane.Markdown("# Bike sales between various age groups in different countries"),
             pn.pane.Markdown("### Bikes are goood exercising machines and is usually good for the older ones."),
             pn.pane.JPG('rheinfelden-bike-store.jpg', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),
             year_slider],
    main=[pn.Row(pn.Column(yaxis_revenue, 
                           revenue_plot.panel(width=700), margin=(0,25)),
                 revenue_table.panel(width=500))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

template.servable();